# Calculating finite difference schemes of arbitrary order for arbitrary kernels

This workbook shows how finite difference schemes can be created for an arbitrary kernel size and derivative order. It is based on Taylor series expansions, following the procedure described in https://web.media.mit.edu/~crtaylor/calculator.html.

A general finite difference scheme for a derivative of order $n$ evaluated around point $x$ with a kernal containing $N$ points

$$ (x + s_1) \Delta x, \: (x + s_2) \Delta x, \: ... , \: (x+s_N) \Delta x$$

is given by

$$ \dfrac{d^{(n)} f}{dx^{(n)}} = \dfrac{1}{\Delta x^n} \left( c_1 f(x + s_1\Delta x) + c_2 f(x + s_2 \Delta x) + ... + c_N f(x + s_N \Delta x) \right) + O(\Delta x^{N-n}). $$

The finite difference coefficients $c_1$, $c_2$, ..., $c_N$ are found by solving the linear system

$$ \left[
\begin{array}{ccc}
s_1^0 & ... & s_N^0 \\
s_1^1 & ... & s_N^1 \\
\vdots & \ddots & \vdots \\
s_1^{N-1} & ... & s_N^{N-1}
\end{array}  \right] 
\left[ \begin{array}{c}
c_1 \\
c_2 \\
\vdots \\
c_N 
\end{array} \right] = 
n!\left[ \begin{array}{c}
\delta_{0,n} \\
\delta_{1,n} \\
\vdots \\
\delta_{N-1,n} 
\end{array} \right]
$$

where $\Delta_{ij}$ is the Kronecker delta function, which is defined as $\delta_{ij} = 1$ for $i=j$ and $\delta_{ij} = 0$ for $i\neq j$.

## Python code

In [13]:
import numpy as np
import fractions as fr

# Sub-routine for the Kronecker delta function
def delta(x,y):
    if x==y:
        return 1
    else:
        return 0

# Sub-routine for building the right-hand-side vector
def rhs(N,d):
    
    l = np.zeros(N,dtype=int)
    
    for i in range(0,N):
        l[i] = np.math.factorial(d)*delta(d,i)
    return l

# Solve for system for the FD coefficients
def FD(s,d):
    
    N = np.size(s)
    l = np.zeros([N,N],dtype=int)
    
    # Build coefficient matrix
    for i in range(0,N):
        l[i,:] = np.power(s,i)
    
    # Build RHS vector
    b = rhs(N,d)
    
    # Solve linear system using NumPy's built-in equation solver
    c = np.linalg.solve(l,b)
    
    return c

### Example 1:

Find the finite difference approximation for $\dfrac{df}{dx}$ with kernel points at $(x-\Delta x, x, x+\Delta x)$:

In [14]:
s = [-1,0,1]
n = 1
stencil_coeffs = FD(s,n)

#Convert to fractions for convenience in comparing with FD tables
print("c=[")
N = np.size(s)
for i in range(0,N):
    print(fr.Fraction(stencil_coeffs[i]).limit_denominator())
print("]")
print('Order of accuracy: O(h ^',N-n,')')

c=[
-1/2
0
1/2
]
Order of accuracy: O(h ^ 2 )


Thus the finite difference approximation is:

$$\dfrac{df}{dx} = \dfrac{1}{\Delta x} \left( -\frac{1}{2} f(x-\Delta x) + \frac{1}{2} f(x+\Delta x)  \right) + O(\Delta x^2)$$

### Example 2:

Find the finite difference approximation for $\dfrac{d^3f}{dx^3}$ with kernel points at $(x-3\Delta x, x-2 \Delta x, x-1 \Delta x, x, x+1\Delta x, x+2 \Delta x, x+3 \Delta x)$:

In [16]:
s = [-3,-2,-1,0,1,2,3]
n = 3
stencil_coeffs = FD(s,n)

#Convert to fractions for convenience in comparing with FD tables
print("c=[")
N = np.size(s)
for i in range(0,N):
    print(fr.Fraction(stencil_coeffs[i]).limit_denominator())
print("]")
print('Order of accuracy: O(h ^',N-n,')')

c=[
1/8
-1
13/8
0
-13/8
1
-1/8
]
Order of accuracy: O(h ^ 4 )


Thus the finite difference approximation is:

$$ \dfrac{d^3f}{dx^3} \approx \dfrac{1}{\Delta x^3} \left( \frac{1}{8}f(x-3\Delta x) - f(x-2\Delta x) + \frac{13}{8} f(x-\Delta x) -\frac{13}{8}f(x+\Delta x) + f(x+2\Delta x) - \frac{1}{8}f(x+3\Delta x) \right) $$

### Comparison to tabulated finite difference coefficients

The finite difference coefficients generated from this workbook can be compared to those available in literature.

For instance, we can reproduce the tables for centered difference approximations from Fornberg, Bengt (1988), "Generation of Finite Difference Formulas on Arbitrarily Spaced Grids", Mathematics of Computation, 51 (184): 699–706, [DOI](dx.doi.org/10.1090/S0025-5718-1988-0935077-0):

| Derivative $n$ | Accuracy | -6 | -5 | -4 | -3 | -2 | -1 | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|----------------|----------|----|----|----|----|----|----|---|---|---|---|---|---|---|
|                | 2        |    |    |    |    |    |-1/2| 0 |1/2|   |   |   |   |   |  
|                | 4        |    |    |    |    |1/12|-2/3|0  |2/3|-1/12| |   |   |   |   
|    First       | 6        |    |    |    |-1/60|3/20|-3/4|0 |3/4|-3/20|1/60||   |   |  
|                | 8        |    |    |1/280|-4/105|1/5|-4/5|0|4/5|-1/5|4/105|-1/280| |   | 
|                | 10       |    |-1/1260|5/504|-5/84|5/21|-5/6|0|5/6|-5/21|5/84|-5/504|1/1260| | 
|                | 12       |1/5544|-1/385|1/56|-5/63|15/56|-6/7|0 | 6/7|-15/56|5/63|-1/56|1/385|-1/5544|

| Derivative $n$ | Accuracy | -6 | -5 | -4 | -3 | -2 | -1 | 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|----------------|----------|----|----|----|----|----|----|---|---|---|---|---|---|---|
|                | 2        |    |    |    |    |    |1| -2 |1|   |   |   |   |   |  
|                | 4        |    |    |    |    |-1/12|4/3|-5/2 |4/3|-1/12| |   |   |   |   
|    Second      | 6        |    |    |    |1/90|-3/20|3/2|49/18 |3/2|-3/20|1/90||   |   |  
|                | 8        |    |    |-1/560|8/315|-1/5|8/5|-205/72|8/5|-1/5|8/315|-1/560| |   | 
|                | 10       |    |1/3150|-5/1008|5/126|-5/21|5/3|-5269/1800|5/3|-5/21|5/126|-5/1008|1/3150| | 

Etc.